In [1]:
import os
import tkinter as tk
from tkinter import messagebox, scrolledtext
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [2]:
# Class 1: Xử lý thuật toán và đồ thị
class Algorithm:
    def __init__(self):
        self.graph = {}

    def create_graph(self, file_path):
        df = pd.read_csv(file_path, sep=r'[;,]', engine='python').iloc[:, :3]
        for _, row in df.iterrows():
            v_from = row['v_from'].strip()
            v_to = row['v_to'].strip()
            weight = float(row['weight'])
            if v_from not in self.graph:
                self.graph[v_from] = {}
            if v_to not in self.graph:
                self.graph[v_to] = {}
            self.graph[v_from][v_to] = weight
            self.graph[v_to][v_from] = weight

    def dijkstra(self, start_node, goal_node):
        distances = {node: float('inf') for node in self.graph}
        distances[start_node] = 0
        parent_nodes = {}
        visited = set()

        while True:
            current_node = None
            current_distance = float('inf')
            for node in distances:
                if node not in visited and distances[node] < current_distance:
                    current_distance = distances[node]
                    current_node = node

            if current_node is None or current_node == goal_node:
                break

            visited.add(current_node)

            for neighbor, weight in self.graph[current_node].items():
                if neighbor not in visited:
                    new_distance = distances[current_node] + weight
                    if new_distance < distances[neighbor]:
                        distances[neighbor] = new_distance
                        parent_nodes[neighbor] = current_node

        return parent_nodes, distances

    def shortest_path(self, parent_nodes, goal_node):
        # Trả về đường đi ngắn nhất dưới dạng danh sách các đỉnh
        path = []
        node = goal_node
        while node in parent_nodes:
            path.insert(0, node)
            node = parent_nodes[node]
        if path:
            path.insert(0, node)
        return path

In [3]:
def dijkstra(self, start_node, goal_node):
        distances = {node: float('inf') for node in self.graph}
        distances[start_node] = 0
        parent_nodes = {}
        visited = set()

        while True:
            current_node = None
            current_distance = float('inf')
            for node in distances:
                if node not in visited and distances[node] < current_distance:
                    current_distance = distances[node]
                    current_node = node

            if current_node is None or current_node == goal_node:
                break

            visited.add(current_node)

            for neighbor, weight in self.graph[current_node].items():
                if neighbor not in visited:
                    new_distance = distances[current_node] + weight
                    if new_distance < distances[neighbor]:
                        distances[neighbor] = new_distance
                        parent_nodes[neighbor] = current_node

        return parent_nodes, distances

In [7]:
# Class 2: Hiển thị giao diện
class App:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Tìm đường đi ngắn nhất - Thuật toán Dijkstra")

        self.algorithm = Algorithm()
        self.start_node = tk.StringVar()
        self.goal_node = tk.StringVar()
        self.radio_buttons = []

        self.create_widgets()
        self.root.mainloop()

    def create_widgets(self):
        # Thanh nhập đường dẫn
        input_fr = tk.Frame(self.root)
        input_fr.pack(side=tk.TOP, fill=tk.X)

        tk.Label(input_fr, text="Đường dẫn file:", font=("Arial", 12, "bold")).pack(side=tk.LEFT, padx=5)
        self.file_path_entry = tk.Entry(input_fr, width=50)
        self.file_path_entry.pack(side=tk.LEFT, padx=5)

        tk.Button(input_fr, text="Vẽ đồ thị", command=self.load_graph).pack(side=tk.LEFT, padx=5)
        tk.Button(input_fr, text="Tìm đường đi", command=self.find_path).pack(side=tk.LEFT, padx=5)

        # Frame hiển thị đồ thị
        self.left_fr = tk.Frame(self.root, bg='white')
        self.left_fr.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.fig, self.ax = plt.subplots(figsize=(6, 6))
        self.ax.axis('off')
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.left_fr)
        self.canvas.get_tk_widget().pack()

        # Frame hiển thị kết quả và chọn đỉnh
        self.right_fr = tk.Frame(self.root, bg='white')
        self.right_fr.pack(side=tk.RIGHT, fill=tk.Y, expand=True)

        # Frame chứa các Label và Radiobutton
        self.selection_frame = tk.Frame(self.right_fr, bg="white") 
        self.selection_frame.pack(side=tk.TOP, fill=tk.X, pady=10)

        # ScrolledText hiển thị kết quả
        self.result_text = scrolledtext.ScrolledText(self.right_fr, width=40, height=20, wrap=tk.WORD)
        self.result_text.configure(font=("Arial", 12, "bold"))
        self.result_text.pack(side=tk.BOTTOM, expand=True)

    def load_graph(self):
        file_path = self.file_path_entry.get().strip().strip('"')
        
        if not os.path.isfile(file_path):
            messagebox.showerror("ERROR", "Kiểm tra lại đường dẫn!!")
            return None
        if not file_path.endswith('.csv'):
            messagebox.showerror("ERROR", "File phải có định dạng .csv!!")
            return None
        if not file_path:
            messagebox.showerror("ERROR", "Không có đường dẫn!!")
            return None

        try:
            self.algorithm.create_graph(file_path)
            # Kiểm tra trọng số
            for node, edges in self.algorithm.graph.items():
                for neighbor, weight in edges.items():
                    if weight < 0:
                        messagebox.showerror("ERROR", "Trọng số các cạnh phải lớn hơn 0 !!")
                        return None
            self.update_node_selection()
            self.result_text.delete('1.0', tk.END)
            self.visualize_graph()

        except Exception as e:
            messagebox.showerror("ERROR", f"Đã xảy ra lỗi khi tải đồ thị: {e}")

    def update_node_selection(self):
        for widget in self.selection_frame.winfo_children():
            widget.destroy()

        # Frame chọn đỉnh bắt đầu
        tk.Label(self.selection_frame, text="Đỉnh bắt đầu:", font=("Arial", 12, "bold"), bg="white").pack(anchor="w", pady=5)
        start_frame = tk.Frame(self.selection_frame, bg="white")
        start_frame.pack(fill=tk.X, pady=5)
        for node in self.algorithm.graph.keys():
            tk.Radiobutton(start_frame, text=node, variable=self.start_node, value=node, bg="white", font=("Arial", 12, "bold")).pack(side=tk.LEFT)

        # Frame chọn đỉnh kết thúc
        tk.Label(self.selection_frame, text="Đỉnh kết thúc:", font=("Arial", 12, "bold"), bg="white").pack(anchor="w", pady=5)
        goal_frame = tk.Frame(self.selection_frame, bg="white")
        goal_frame.pack(fill=tk.X, pady=5)
        for node in self.algorithm.graph.keys():
            tk.Radiobutton(goal_frame, text=node, variable=self.goal_node, value=node, bg="white", font=("Arial", 12, "bold")).pack(side=tk.LEFT)

    def visualize_graph(self, path=None):
        G = nx.Graph()
        for node, edges in self.algorithm.graph.items():
            for neighbor, weight in edges.items():
                G.add_edge(node, neighbor, weight=weight)

        if not hasattr(self, "pos") or path is None:  # Chỉ tạo bố cục mới nếu chưa có hoặc không cần tô đường đi
            self.pos = nx.spring_layout(G)  # Lưu bố cục đồ thị

        edge_labels = nx.get_edge_attributes(G, 'weight')

        self.ax.clear()
        nx.draw(G, self.pos, with_labels=True, node_color="thistle", edge_color="black", ax=self.ax, font_size=13)
        nx.draw_networkx_nodes(G, self.pos, node_size=800, node_color="thistle", edgecolors="black", ax=self.ax, )
        nx.draw_networkx_edge_labels(G, self.pos, edge_labels=edge_labels, ax=self.ax, font_color="black", font_size=12)

        if path:  # Tô đỏ các cạnh trong đường đi
            path_edges = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
            nx.draw_networkx_nodes(G, self.pos, nodelist=path, node_color="orangered", edgecolors="black", node_size=800, ax=self.ax)
            nx.draw_networkx_edges(G, self.pos, edgelist=path_edges, edge_color="orangered", width=2.5, ax=self.ax)
            red_edge_labels = {}
            for edge in path_edges:
                if edge in edge_labels:
                    red_edge_labels[edge] = edge_labels[edge]
                elif (edge[1], edge[0]) in edge_labels:
                    red_edge_labels[(edge[1], edge[0])] = edge_labels[(edge[1], edge[0])]
            nx.draw_networkx_edge_labels(G, self.pos, edge_labels=red_edge_labels, font_color="orangered", ax=self.ax, font_size=12)

        self.canvas.draw()

    def find_path(self):
        start = self.start_node.get()
        goal = self.goal_node.get()

        if not start or not goal:
            messagebox.showerror("ERROR", "Phải chọn cả đỉnh bắt đầu và đỉnh kết thúc!")
            return None
        if start == goal:
            messagebox.showerror("ERROR", "Đỉnh bắt đầu và đỉnh kết thúc phải khác nhau!!")
            return None

        try:
            parent_nodes, distances = self.algorithm.dijkstra(start, goal)
            path = self.algorithm.shortest_path(parent_nodes, goal)

            if not path:
                self.result_text.delete('1.0', tk.END)
                self.result_text.insert('1.0', f"\n\nKhông tìm thấy đường đi từ {start} đến {goal}.\n")
            else:
                self.result_text.delete('1.0', tk.END)
                self.result_text.insert('1.0', f"\nĐường đi ngắn nhất: {' -> '.join(path)}\n\nKhoảng cách: {distances[goal]:.2f}\n")
                self.visualize_graph(path=path)

        except KeyError as e:
            messagebox.showerror("ERROR", f"Đỉnh {e} không tồn tại trong đồ thị.")
        except Exception as e:
            messagebox.showerror("ERROR", f"Đã xảy ra lỗi: {e}")

In [ ]:
if __name__ == "__main__":
    App()